In [145]:
from helper_load import get_engine_database
from collections import defaultdict
import pandas as pd

with open("./secrets.txt", "r") as f:
    driver, username, password, host, port, database = f.read().split(",")
engine = get_engine_database(driver, username, password, host, port, database)

In [146]:
socios = pd.read_excel("Empresas-Sócios_CNAE-9602501_Cidade-Maringa_test_1.xlsx", sheet_name="Sócios")
socios.head()

,CNPJ Básico,Identificador Sócio,Nome Sócio,CNPJ/CPF Sócio,Qualificação Sócio,Data Entrada Sociedade,País,CPF Representante Legal,Nome Representante Legal
0,34215594,PESSOA FISICA,RONALDO BALEEIRO VIOTTO,***477459**,Sócio,28/07/2022,NAO DECLARADOS,***000000**,NaN
1,34215594,PESSOA FISICA,STEFANIE CAMILA DE OLIVEIRA MALTA,***612599**,Sócio-Administrador,28/07/2022,NAO DECLARADOS,***000000**,NaN
2,32803268,PESSOA FISICA,ABEL RODRIGUES,***122309**,Sócio,18/02/2019,NAO DECLARADOS,***000000**,NaN
3,38143484,PESSOA FISICA,ISABELA LAHUTT ANTONIO,***399809**,Sócio-Administrador,19/08/2020,NAO DECLARADOS,***000000**,NaN
4,24680078,PESSOA FISICA,ALINE DE ARRUDA GARCIA DOS SANTOS,***602509**,Sócio-Administrador,28/04/2016,NAO DECLARADOS,***000000**,NaN


In [147]:
def get_data(nomes: list[str], cnpjs_cpfs: list[str], telefones_coletados: list[str]=[]) -> tuple[dict, set]:
    socio_numero_dict = defaultdict(str)
    for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
        with engine.begin() as conn:
            qry = f"""SELECT telefone FROM public.socio_telefone
            WHERE nome_socio = '{nome}' AND cnpj_cpf_socio = '{cnpj_cpf}' AND telefone <> ' ';"""

            socio_numero_dict[f"{nome}-{cnpj_cpf}"] = pd.read_sql(qry, conn).dropna().drop_duplicates()["telefone"].values[0]

    todos_telefones = set()
    for _, t in dict(socio_numero_dict).items():
        todos_telefones.add(t)

    if telefones_coletados:
        return dict(socio_numero_dict), set([t for t in todos_telefones if t not in telefones_coletados])
    
    return dict(socio_numero_dict), todos_telefones

In [148]:
nomes, cnpjs_cpfs = socios["Nome Sócio"], socios["CNPJ/CPF Sócio"]

socio_numero_dict, todos_telefones = get_data(nomes, cnpjs_cpfs)
print(socio_numero_dict)
print(len(todos_telefones), todos_telefones)

{'RONALDO BALEEIRO VIOTTO-***477459**': '44 98612552', 'STEFANIE CAMILA DE OLIVEIRA MALTA-***612599**': '44 98612552', 'ABEL RODRIGUES-***122309**': '43 98311055', 'ISABELA LAHUTT ANTONIO-***399809**': '44 99380082', 'ALINE DE ARRUDA GARCIA DOS SANTOS-***602509**': '44 30282991', 'VERA LUCIA DE ARRUDA GARCIA-***726549**': '44 30282991', 'VAGNER JOAQUIM FERREIRA DA SILVA-***437310**': '44 33464405', 'GENIVALDO RIBEIRO DA SILVA-***164351**': '18 32232860', 'FABRICIA BRASILINO DE SOUZA RODRIGUES-***576729**': '43 98311055', 'ALEXANDRE LEONARDO DE LIMA COUTINHO-***685609**': '11 55064059', 'VILSON GUERRA DOS SANTOS-***156589**': '44 30253626', 'RENATO BOSCARDIM SOARES-***734389**': '44 99335807', 'ALESSANDRO DE OLIVEIRA ROMAN-***056199**': '44 91335565', 'JOICE LOANA OLIANO DOS SANTOS-***201729**': '44 88444039', 'VANESSA VERAS VIEIRA-***195078**': '18 32033950', 'RAISSA BENTO SANTOS-***071344**': '44 33057778', 'MARCOS VINICIUS NERONI FERNANDES-***528291**': '18 32033950', 'LAIS RUSSO PAI

In [ ]:
def get_socios_telefones(nomes: list[str],
                         cnpjs_cpfs: list[str],
                         socio_numero_dict: dict[str],
                         todos_telefones: set,
                         socios_telefones: list[tuple] = []) -> tuple[list[tuple], list[tuple]]:
    socios_telefones_replace = []
    for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
        ts = socio_numero_dict[f"{nome}-{cnpj_cpf}"]
        if ts in todos_telefones:
            socios_telefones.append((nome, cnpj_cpf, ts))
            todos_telefones.remove(ts)
        else:
            socios_telefones_replace.append((nome, cnpj_cpf))
    return socios_telefones, socios_telefones_replace

In [128]:
socios_telefones, socios_telefones_replace

socios_telefones = []
socios_telefones_replace = []
for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
    ts = socio_numero_dict[f"{nome}-{cnpj_cpf}"]
    if ts in todos_telefones:
        socios_telefones.append((nome, cnpj_cpf, ts))
        todos_telefones.remove(ts)
    else:
        socios_telefones_replace.append((nome, cnpj_cpf))

In [132]:
socios_telefones_replace, len(socios_telefones_replace)

([('STEFANIE CAMILA DE OLIVEIRA MALTA', '***612599**'),
  ('VERA LUCIA DE ARRUDA GARCIA', '***726549**'),
  ('FABRICIA BRASILINO DE SOUZA RODRIGUES', '***576729**'),
  ('MARCOS VINICIUS NERONI FERNANDES', '***528291**'),
  ('TIAGO FOGACA CRESPO', '***858739**'),
  ('SIDERLEY FREITAS DE CASTRO', '***142339**'),
  ('SIMONE DA LUZ BARBOSA', '***009371**'),
  ('ROSIMEIRE COLOGNESE DOS SANTOS ROZAO', '***885459**'),
  ('JAQUELINE RUUS DE SOUZA', '***741079**'),
  ('JAQUELINE MARIA GENERAL MARQUES KLOSOSKI', '***523809**'),
  ('MATEUS RIBEIRO ALVES', '***222109**'),
  ('MATEUS UBIALI CINTRA', '***460429**'),
  ('JOAO ROBERTO FRAGUAS', '***300119**'),
  ('FELIPE MUNIZ DA SILVA', '***451299**'),
  ('ELAINE APARECIDA DE MATTOS', '***989689**'),
  ('JULIANO SICHIERI GUILHEN', '***749479**'),
  ('JEAN RICARDO SOARES SANT ANA', '***191299**')],
 17)

In [131]:
socio_numero_dict = defaultdict(str)
nomes, cnpjs_cpfs = [i[0] for i in socios_telefones_replace], [i[1] for i in socios_telefones_replace]
for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
    with engine.begin() as conn:
        qry = f"""SELECT
        telefone
    FROM public.socio_telefone
    WHERE nome_socio = '{nome}' AND cnpj_cpf_socio = '{cnpj_cpf}' AND telefone <> ' ';"""

        socio_numero_dict[f"{nome}-{cnpj_cpf}"] = pd.read_sql(qry, conn).dropna().drop_duplicates()["telefone"].values[0]

todos_telefones = set()
for _, t in dict(socio_numero_dict).items():
    todos_telefones.add(t)

reduced_todos_telefones = [t for t in todos_telefones if t not in [i[-1] for i in socios_telefones]]

socio_numero_dict = dict(socio_numero_dict)
print(socio_numero_dict)
print(len(reduced_todos_telefones), reduced_todos_telefones)

{'STEFANIE CAMILA DE OLIVEIRA MALTA-***612599**': '44 98612552', 'VERA LUCIA DE ARRUDA GARCIA-***726549**': '44 30282991', 'FABRICIA BRASILINO DE SOUZA RODRIGUES-***576729**': '14 32321473', 'MARCOS VINICIUS NERONI FERNANDES-***528291**': '18 32232860', 'TIAGO FOGACA CRESPO-***858739**': '44 30267003', 'SIDERLEY FREITAS DE CASTRO-***142339**': '44 30319488', 'SIMONE DA LUZ BARBOSA-***009371**': '44 33057334', 'ROSIMEIRE COLOGNESE DOS SANTOS ROZAO-***885459**': '44 99944577', 'JAQUELINE RUUS DE SOUZA-***741079**': '44 97436323', 'JAQUELINE MARIA GENERAL MARQUES KLOSOSKI-***523809**': '44 97436323', 'MATEUS RIBEIRO ALVES-***222109**': '44 33052768', 'MATEUS UBIALI CINTRA-***460429**': '44 30267003', 'JOAO ROBERTO FRAGUAS-***300119**': '44 88295015', 'FELIPE MUNIZ DA SILVA-***451299**': '44 30267003', 'ELAINE APARECIDA DE MATTOS-***989689**': '44 30310263', 'JULIANO SICHIERI GUILHEN-***749479**': '44 98565985', 'JEAN RICARDO SOARES SANT ANA-***191299**': '44 30285353'}
2 ['14 32321473', '

In [133]:
socios_telefones_replace = []
for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
    ts = socio_numero_dict[f"{nome}-{cnpj_cpf}"]
    if ts in reduced_todos_telefones:
        socios_telefones.append((nome, cnpj_cpf, ts))
        reduced_todos_telefones.remove(ts)
    else:
        socios_telefones_replace.append((nome, cnpj_cpf))

In [134]:
socios_telefones_replace, len(socios_telefones_replace)

([('STEFANIE CAMILA DE OLIVEIRA MALTA', '***612599**'),
  ('VERA LUCIA DE ARRUDA GARCIA', '***726549**'),
  ('TIAGO FOGACA CRESPO', '***858739**'),
  ('SIDERLEY FREITAS DE CASTRO', '***142339**'),
  ('SIMONE DA LUZ BARBOSA', '***009371**'),
  ('ROSIMEIRE COLOGNESE DOS SANTOS ROZAO', '***885459**'),
  ('JAQUELINE RUUS DE SOUZA', '***741079**'),
  ('JAQUELINE MARIA GENERAL MARQUES KLOSOSKI', '***523809**'),
  ('MATEUS RIBEIRO ALVES', '***222109**'),
  ('MATEUS UBIALI CINTRA', '***460429**'),
  ('JOAO ROBERTO FRAGUAS', '***300119**'),
  ('FELIPE MUNIZ DA SILVA', '***451299**'),
  ('ELAINE APARECIDA DE MATTOS', '***989689**'),
  ('JULIANO SICHIERI GUILHEN', '***749479**'),
  ('JEAN RICARDO SOARES SANT ANA', '***191299**')],
 15)

```
([('STEFANIE CAMILA DE OLIVEIRA MALTA', '***612599**'),
  ('VERA LUCIA DE ARRUDA GARCIA', '***726549**'),
  ('TIAGO FOGACA CRESPO', '***858739**'),
  ('SIDERLEY FREITAS DE CASTRO', '***142339**'),
  ('SIMONE DA LUZ BARBOSA', '***009371**'),
  ('ROSIMEIRE COLOGNESE DOS SANTOS ROZAO', '***885459**'),
  ('JAQUELINE RUUS DE SOUZA', '***741079**'),
  ('JAQUELINE MARIA GENERAL MARQUES KLOSOSKI', '***523809**'),
  ('MATEUS RIBEIRO ALVES', '***222109**'),
  ('MATEUS UBIALI CINTRA', '***460429**'),
  ('JOAO ROBERTO FRAGUAS', '***300119**'),
  ('FELIPE MUNIZ DA SILVA', '***451299**'),
  ('ELAINE APARECIDA DE MATTOS', '***989689**'),
  ('JULIANO SICHIERI GUILHEN', '***749479**'),
  ('JEAN RICARDO SOARES SANT ANA', '***191299**')],
 15)
```